In [19]:
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
# import torch.nn as nn 
import time
import math
# import torch.nn.functional as F
from net import *
from utils import *
from tracker import *
import onnx


In [20]:
backbone_m = Inception3_M()
backbone_m.update_params()
backbone_q = Inception3_Q()
backbone_q.update_params()
neck_m = AdjustLayer()
neck_m.update_params()
neck_q = AdjustLayer()
neck_q.update_params()
head = STMHead()
head.update_params()

model = STMTrack(backbone_m, backbone_q, neck_m, neck_q, head)
# model.update_params()

# Convert BatchNorm to SyncBatchNorm 
# task_model = convert_model(task_model)
model_file = "new-epoch-19.pkl"

# model_file = "epoch-19.pkl"
model_state_dict = torch.load(model_file,
                        map_location=torch.device("cpu"))

# model.load_state_dict(model_state_dict['model_state_dict'])
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [21]:
net = model.basemodel_q

In [22]:
ONNX_FILE_PATH = "backbone_q.onnx"

In [23]:
input = torch.randn(1,3,289,289).cuda()
net.eval()
net.cuda()
torch.onnx.export(net, input, ONNX_FILE_PATH, input_names=["query_img"], output_names=["h_q"], export_params=True)

In [24]:
class Memorize(nn.Module):
    def __init__(self):
        super(Memorize, self).__init__()
        self.backbone_m = model.basemodel_m
        self.neck_m = model.neck_m
    def forward(self, im_crop, fg_bg_label_map):
    
        fm = self.backbone_m(im_crop, fg_bg_label_map)
        fm = self.neck_m(fm)
        fm = fm.permute(1, 0, 2, 3).unsqueeze(0).contiguous()  # B, C, T, H, W
        return fm

In [25]:
net = Memorize()#(model.basemodel_m,model.neck_m)

In [26]:
ONNX_FILE_PATH = "memorize.onnx"
input = torch.randn(1,3,289,289).cuda()
fg_bg = torch.randn(1,1,289,289).cuda()
net.eval()
net.cuda()
torch.onnx.export(net, (input,fg_bg), ONNX_FILE_PATH, input_names=["img","fg_bg_label_map"], output_names=["fm"], export_params=True)

In [27]:
fm = net(input,fg_bg)

In [28]:
fm.shape

torch.Size([1, 512, 1, 25, 25])

In [29]:
512*25*25

320000

In [30]:
class Head(nn.Module):
    def __init__(self,head):
        super(Head, self).__init__()
        self.head = head
    def forward(self,fm,fq):
        q_si ze = 289
        y = self.head.memory_read(fm, fq)
        cls_score, ctr_score, offsets = self.head.solve(y)
        return cls_score, ctr_score, offsets

In [31]:
net = Head(model.head)

In [32]:
ONNX_FILE_PATH = "head.onnx"
fm = torch.randn(1, 512, 6, 25, 25).cuda()
fq = torch.randn(1, 512, 25, 25).cuda()
net.eval()
net.cuda()



Head(
  (head): STMHead(
    (cls_ctr): ConvModule(
      (conv1): Conv2d(1024, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv_layers): Sequential(
        (0): ReLU(inplace=True)
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (10): ReLU(inplace=True)
        (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (reg): ConvModule(
      (conv1): Conv2d(1024, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv_layers): Sequentia

In [33]:
cls_score, ctr_score, offsets = net(fm,fq)

In [34]:
torch.onnx.export(net, (fm,fq), 
                  ONNX_FILE_PATH, input_names=["fm","fq"], 
                  output_names=["cls_score", "ctr_score", "offsets"], export_params=True)

In [35]:
cls_score.shape

torch.Size([1, 1, 25, 25])

In [36]:
ctr_score.shape

torch.Size([1, 1, 25, 25])

In [37]:
offsets.shape

torch.Size([1, 4, 25, 25])

In [42]:
net = Head(model.head)
fm = torch.ones(1, 512, 6, 25, 25).cuda()
fq = torch.ones(1, 512, 25, 25).cuda()

In [43]:
cls_score, ctr_score, offsets = net(fm,fq)

In [51]:
cls_score_cpp = []
with open("trt/build/cls_score.txt") as f:
    cls_score_cpp = f.readlines()
    


In [54]:
cls_score_cpp = [float(c[:-1]) for c in cls_score_cpp]

In [62]:
cls_score_cpp = np.array(cls_score_cpp)

In [63]:
cls_score = cls_score.flatten().cpu().detach().numpy()

In [69]:
np.abs(cls_score_cpp-cls_score).max()

0.015470562667846632

In [70]:
model.head.si

Parameter containing:
tensor(0.7557, device='cuda:0', requires_grad=True)

In [71]:
model.head.bi

Parameter containing:
tensor(1.7903, device='cuda:0', requires_grad=True)